## Introduction

**Loans dataset aims to identify variables which indicate if a person is likely to default, which can be used for identifying the risky loan applicants to avoid any financial loss to the company.**

![alt text](15000-loan-1.jpg "Title")


**Let's solve the below questions to analyze the characteristics of loan borrowers and identify the risky applicants** 

## Dataset Description
This dataset contains the complete loan data for all loans issued through the time period 2007 to 2011.

**Data Dictionary -** <br>
**1.annual_inc -** The self-reported annual income provided by the borrower during registration.<br>
**2.dti -** A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self reported monthly income.<br>
**3.emp_length -** Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.<br>
**4.funded_amnt -** The total amount committed to that loan at that point in time.<br>
**5.funded_amnt_inv -** The total amount committed by investors for that loan at that point in time.<br>
**6.grade -** LC assigned loan grade<br>
**7.id -** A unique LC assigned ID for the loan listing.<br>
**8.installment -** The monthly payment owed by the borrower if the loan originates.<br>
**9.int_rate -** Interest Rate on the loan. <br>
**10.last_pymnt_amnt-** Last total payment amount received. <br>
**11.last_pymnt_d -** Last month payment was received<br>
**12.loan_amnt -** The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.<br>
**13.loan_status -** Current status of the loan<br>
**14.member_id -** A unique LC assigned Id for the borrower member.<br>
**15.purpose -** A category provided by the borrower for the loan request.<br>
**16.term -** The number of payments on the loan. Values are in months and can be either 36 or 60.<br>
**17.total_acc -** The total number of credit lines currently in the borrower's credit file<br>
**18.total_pymnt -** Payments received to date for total amount funded<br>
**19.total_pymnt_inv -** Payments received to date for portion of total amount funded by investors<br>
**20.total_rec_int -** Interest received to date

In [1]:
import pandas as pd
import numpy as np

### 1. Import the necessary packages and load the `loan.csv` dataset

In [3]:
df=pd.read_csv('loan.csv')

### 2. Check the shape of the dataset - print the number of rows and columns

In [4]:
df.shape

(39717, 23)

### 3. Check the data type and info

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   39717 non-null  int64  
 1   member_id            39717 non-null  int64  
 2   loan_amnt            39717 non-null  int64  
 3   funded_amnt          39717 non-null  int64  
 4   funded_amnt_inv      39717 non-null  float64
 5   term                 39717 non-null  object 
 6   int_rate             39717 non-null  object 
 7   installment          39717 non-null  float64
 8   grade                39717 non-null  object 
 9   emp_length           38642 non-null  object 
 10  annual_inc           39717 non-null  float64
 11  verification_status  39717 non-null  object 
 12  loan_status          39717 non-null  object 
 13  purpose              39717 non-null  object 
 14  dti                  39717 non-null  float64
 15  total_pymnt          39717 non-null 

### 4. Drop the Unnamed: 21 and Unnamed: 22 columns

In [8]:
df.drop(columns=['Unnamed: 21','Unnamed: 22'], inplace=True)

### 5. The column int_rate is character type, let's convert it to float. It contains "%" so we will have to remove the "%" sign first.

In [13]:
df['int_rate']=df['int_rate'].str[:-1].astype('float')

### 6. Check if there are any duplicate member_id

In [12]:
df.duplicated(subset='member_id').sum()

0

### 7. Write the code to find the value counts of the ‘loan_status’ category column and filter only the ‘fully paid’ and ‘charged off’ categories.

In [26]:
df['loan_status'].value_counts([['Fully Paid','Charged off']])

loan_status
Fully Paid     0.829620
Charged Off    0.141677
Current        0.028703
Name: proportion, dtype: float64

### 8. From ‘Emp_Len’ column extract the numerical value from the string. It might be possible that it has null values (drop the missing values)
Hint - Emp_len : < 1year, 2 years , 3 years as 1 , 2, 3 so on.

In [31]:
df.dropna(subset='emp_length',inplace=True)

In [32]:
df['emp_length'].isnull().sum()

0

In [42]:
def no_extract(a):
    res=''
    for i in a:
        if i.isdigit():
            res+=i
    return res
#no_extract('10 yea')

In [49]:
a='10 ykjd'
(lambda a:''.join([i for i in a if i.isdigit()]))(a)

'10'

In [ ]:
df['emp_length']=df['emp_length'].map(no_extract).astype('int')

In [44]:
df['emp_length'].dtype

dtype('int32')

### 9. Using the Lambda function, remove the month from the ‘term’ column such that ‘36 months’, ‘60 months’ appear as 36 and 60 respectively.

In [50]:
df['term']=df['term'].map(lambda a:''.join([i for i in a if i.isdigit()])).astype('int')

In [51]:
df['term'].dtype

dtype('int32')

### 10.Find the count of applicants whose 
A. loan_amnt and funded_amnt are same.<br>
B. loan_amnt is more than funded_amnt.<br>
C. loan_amnt is less than funded_amnt<br>

In [56]:
print(df.query('loan_amnt==funded_amnt').shape[0])
print(df.query('loan_amnt>funded_amnt').shape[0])
print(df.query('loan_amnt<funded_amnt').shape[0])

36826
1816
0


### 11.Using a user defined function convert the ‘emp_len’ column into categorical column as follows -
If emp_len is less than equals to 1 then recode as ‘fresher’.<br>
If emp_len is greater than 1 and less than 3 then recode as ‘junior’.<br>
If emp_len is greater than 3 and less than 7 then recode as ‘senior’<br>
If emp_len is greater than 7 then recode as ‘expert’.<br>

Note - It might be possible that this column contains null value so check and drop the null values

In [65]:
df['emp_length']=df['emp_length'].map(
(lambda x:'Fresher' if x<=1 else 'Junior' if x<3 else 'Senior' if x<7 else 'Expert'))

In [67]:
df['emp_length']

0         Expert
1        Fresher
2         Expert
3         Expert
4        Fresher
          ...   
39712     Senior
39713     Senior
39714    Fresher
39715    Fresher
39716    Fresher
Name: emp_length, Length: 38642, dtype: object

### 12. `verification_status` column conatins ['Verified', 'Source Verified', 'Not Verified'] convert 'Verified' and 'Source Verified' into single category as varified and keep not verified as it is.

In [68]:
df['verification_status'].unique()
df.loc[df['verification_status']!='Not Verified','verification_status'] ='Verified'
df['verification_status'].unique()

array(['Verified', 'Not Verified'], dtype=object)

### 13. For each grade find the verified and not verified applicants count. Rename the column as Not Verified Count, Verified Count

In [70]:
df_13=df.pivot_table(index='grade',columns='verification_status',values='member_id',aggfunc='count')
df_13.rename(columns={'Not Verified': 'Not Verified count', 'Verified':'Verified count'}, inplace=True)
df_13

verification_status,Not Verified count,Verified count
grade,,
A,5006,4689
B,5277,6409
C,3435,4491
D,1918,3277
E,648,2147
F,170,860
G,44,271


### 14.Find the min interest rate, max interest rate, average interest rate for each grade under sub category of Verification_status use pivot_table. Change the multiIndex into single level column joined with "_".

In [71]:
df_14=df.pivot_table(index='grade',columns='verification_status',values='int_rate',aggfunc=['min','max','mean'])
df_14

min                   max                  mean  \
verification_status Not Verified Verified Not Verified Verified Not Verified   
grade                                                                          
A                           5.42     5.42         9.63     9.63     7.413096   
B                           6.00     6.00        12.69    12.69    10.979873   
C                           6.00    11.03        16.11    16.11    13.346815   
D                           6.00     6.00        18.49    18.49    15.278978   
E                          13.43     6.00        20.30    20.99    17.043117   
F                          15.01    15.76        22.64    22.94    18.793176   
G                          17.34    18.84        24.40    24.59    20.828864   

                                
verification_status   Verified  
grade                           
A                     7.287426  
B                    11.070037  
C                    13.700891  
D                    15.963872  
E                    17.907103  
F                    19.928930  
G                    21.491328

In [72]:
df_14.columns=[i[1]+'_int_rate_'+i[0] for i in df_14.columns]
df_14

,Not Verified_int_rate_min,Verified_int_rate_min,Not Verified_int_rate_max,Verified_int_rate_max,Not Verified_int_rate_mean,Verified_int_rate_mean
grade,,,,,,
A,5.42,5.42,9.63,9.63,7.413096,7.287426
B,6.00,6.00,12.69,12.69,10.979873,11.070037
C,6.00,11.03,16.11,16.11,13.346815,13.700891
D,6.00,6.00,18.49,18.49,15.278978,15.963872
E,13.43,6.00,20.30,20.99,17.043117,17.907103
F,15.01,15.76,22.64,22.94,18.793176,19.928930
G,17.34,18.84,24.40,24.59,20.828864,21.491328


### 15. Join the table created in question 13 and 14 on grade column.

In [73]:
df_13.join(df_14)

,Not Verified count,Verified count,Not Verified_int_rate_min,Verified_int_rate_min,Not Verified_int_rate_max,Verified_int_rate_max,Not Verified_int_rate_mean,Verified_int_rate_mean
grade,,,,,,,,
A,5006,4689,5.42,5.42,9.63,9.63,7.413096,7.287426
B,5277,6409,6.00,6.00,12.69,12.69,10.979873,11.070037
C,3435,4491,6.00,11.03,16.11,16.11,13.346815,13.700891
D,1918,3277,6.00,6.00,18.49,18.49,15.278978,15.963872
E,648,2147,13.43,6.00,20.30,20.99,17.043117,17.907103
F,170,860,15.01,15.76,22.64,22.94,18.793176,19.928930
G,44,271,17.34,18.84,24.40,24.59,20.828864,21.491328


### 16.Find the average loan_amnt and average interest rate for each grade under sub category of purpose.

In [74]:
df_16=df.pivot_table(index='grade',columns='purpose',values=['loan_amnt','int_rate'],aggfunc='mean')
df_16

int_rate                                             \
purpose        car credit_card debt_consolidation educational   
grade                                                           
A         7.153068    7.460133           7.362349    8.356456   
B        10.831100   11.016812          11.061365   11.260408   
C        13.594523   13.499896          13.591584   12.928444   
D        15.803281   15.702302          15.739319   14.745161   
E        17.083750   17.704337          17.795702   16.326429   
F        19.107273   19.687000          19.805136   17.245000   
G        21.255000   21.676207          21.513706   21.270000   

                                                                          \
purpose home_improvement      house major_purchase    medical     moving   
grade                                                                      
A               7.228232   7.480989       7.105088   7.244518   7.436585   
B              10.995603  10.973714      10.967449  10.922690  11.009830   
C              13.579155  13.687000      13.463840  13.534135  13.669554   
D              15.676644  15.960784      15.553142  15.932651  15.612154   
E              17.851421  17.680000      17.482041  17.016585  17.787143   
F              20.014667  19.573333      18.971538  19.487857  19.285556   
G              21.205000  21.448333      21.985000  20.620000  21.038000   

                    ...        loan_amnt                               \
purpose      other  ... home_improvement         house major_purchase   
grade               ...                                                 
A         7.497885  ...      8959.458735  10072.252747    6714.290201   
B        11.084367  ...     12191.590126  13516.428571    9121.130952   
C        13.461817  ...     11790.115163  13511.785714    7737.066667   
D        15.693992  ...     12833.220339  14033.823529    8713.163717   
E        17.772172  ...     16712.021858  15221.666667   11915.306122   
F        19.822317  ...     17884.166667  17356.666667   17795.192308   
G        21.806190  ...     22495.312500  21816.666667   16783.333333   

                                                                    \
purpose       medical        moving         other renewable_energy   
grade                                                                
A         7471.319797   6521.036585   6868.104288      7729.310345   
B         7561.548223   6979.687500   8291.279070      9321.875000   
C         8592.105263   5956.919643   7562.635379      6940.000000   
D         8279.216867   5959.230769   8743.788187     11966.666667   
E        10726.829268   8875.000000  10776.244344      6130.000000   
F        17701.785714   6033.333333  14484.146341     15416.666667   
G        18733.333333  18585.000000  17452.380952     18500.000000   

                                                   
purpose small_business     vacation       wedding  
grade                                              
A          9497.230539  5603.483607   8391.317992  
B         13653.131749  5986.344538   9885.714286  
C         14067.169540  4448.333333   9893.250000  
D         14802.881356  5398.214286  10277.777778  
E         16141.293532  6465.000000  14498.584906  
F         15124.210526  9000.000000  13400.000000  
G         17068.085106  6600.000000  11766.666667  

[7 rows x 28 columns]

### 17.Under each grade which purpose has the highest average interest rates?

In [79]:
df_16['int_rate'].idxmax(axis=1)

grade
A         educational
B         educational
C               house
D    renewable_energy
E    home_improvement
F    home_improvement
G    renewable_energy
dtype: object

In [80]:
# Filter the data
filtered_data = df[(df['purpose'] == 'credit_card') & (df['grade'] == 'A')]

# Calculate the mean interest rate
mean_int_rate = filtered_data['int_rate'].mean()

print("Mean Interest Rate for Credit Card Purpose in Grade A:", mean_int_rate)


Mean Interest Rate for Credit Card Purpose in Grade A: 7.460133037694014
